In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("train.csv")

In [3]:
df.shape

(404290, 6)

In [4]:
df

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0
...,...,...,...,...,...,...
404285,404285,433578,379845,How many keywords are there in the Racket prog...,How many keywords are there in PERL Programmin...,0
404286,404286,18840,155606,Do you believe there is life after death?,Is it true that there is life after death?,1
404287,404287,537928,537929,What is one coin?,What's this coin?,0
404288,404288,537930,537931,What is the approx annual cost of living while...,I am having little hairfall problem but I want...,0


In [5]:
new_df = df.sample(30000)

In [6]:
new_df.isnull().sum()

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

In [7]:
new_df['question2'].fillna(new_df['question2'].mode()[0], inplace=True)

In [8]:
new_df.duplicated().sum()

0

In [9]:
new_df.isnull().sum()

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

In [10]:
que_df = new_df[['question1','question2']]

In [11]:
que_df

,question1,question2
45267,Is it just me or does Chris Everett look like ...,Who has cloned the clone of a clone?
251759,How do I get a good rank in the GATE exam to g...,How do I prepare to get a good rank in the GAT...
1232,What is the the best free wordpress theme for ...,"Under either candidate as president, is the US..."
219200,How do you know when hard boiled eggs are done?,Are soft boiled eggs safe?
63943,Why is the NDTV news channel banned for one da...,Why was NDTV banned for 24 hours?
...,...,...
337170,What is Uniform Civil Code? What are the benef...,What will be the benefits of uniform civil code?
312424,What is Batman's personality like?,What is Batman all about?
314443,Is porn addictive?,Is porn addiction real?
149600,How do I make a website like cafecuba.in?,How do I make websites?


In [12]:
from sklearn.feature_extraction.text import CountVectorizer

In [13]:
questions = list(que_df['question1']) + list(que_df['question2'])
cv = CountVectorizer(max_features=3000)

In [14]:
q1_arr, q2_arr = np.vsplit(cv.fit_transform(questions).toarray(),2)

In [15]:
temp_df1 = pd.DataFrame(q1_arr, index = que_df.index)
temp_df2 = pd.DataFrame(q2_arr, index = que_df.index)

In [16]:
temp_df = pd.concat([temp_df1, temp_df2], axis=1)
temp_df.shape

(30000, 6000)

In [17]:
temp_df['is_duplicate'] = new_df['is_duplicate']

In [18]:
temp_df.head()

,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,is_duplicate
45267,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
251759,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1232,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
219200,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
63943,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(temp_df.iloc[:,0:-1].values, temp_df.iloc[:,-1].values, test_size=0.2, random_state =100)

In [20]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test, y_pred)

0.7376666666666667

In [21]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)
accuracy_score(y_test,y_pred)

0.7266666666666667